In [1]:
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

In [8]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
open_category = open_category.loc[(open_category['parent'].notnull()),:]
open_category = open_category.rename(columns={'slug':'global_slug','parent':'brand_slug','name':'global_name'})
open_category = open_category.loc[:,['brand_slug','global_slug','global_name']]
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
open_model_detail = open_model_detail.merge(open_category, how='left', on=['global_slug'])
open_model_detail.head()

,id,price_bn,global_slug,year,volume,control,detail_model,model_detail_slug,brand_slug,global_name
0,96,81.80,audi-a7,2012,3.0,自动,2012款 3.0TFSI S-Tronic quattro 舒适型,model_25023_cs,audi,奥迪A7
1,97,95.80,audi-a7,2012,3.0,自动,2012款 3.0TFSI S-Tronic quattro 豪华型,model_15554_cs,audi,奥迪A7
2,120,48.98,i-audi-a4,2013,2.0,自动,2013款 2.0T 40 TFSI allroad quattro 舒适版,model_28145_cs,audi,奥迪A4(进口)
3,121,51.98,i-audi-a4,2013,2.0,自动,2013款 2.0T 40 TFSI allroad quattro 豪华版,model_26127_cs,audi,奥迪A4(进口)
4,161,345.00,astonmartin-db9,2011,6.0,自动,2011款 6.0L 自动 Touchtronic Volante,model_25573_cs,astonmartin,阿斯顿马丁DB9


In [12]:
test = open_model_detail.loc[(open_model_detail['brand_slug'] == 'dazhong'),:]
test['title'] = test['global_name'] + ' ' + test['detail_model']
test['predict_brand_slug'] = test['brand_slug']
test.to_csv('../tmp/train/predict.csv', index=False)
test.head()

/home/ml/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ml/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,price_bn,global_slug,year,volume,control,detail_model,model_detail_slug,brand_slug,global_name,title,predict_brand_slug
91,1448,10.78,baolai,2013,1.6,手动,2013款 1.6L 手动 时尚型,model_31544_cs,dazhong,大众宝来,大众宝来 2013款 1.6L 手动 时尚型,dazhong
92,1449,11.63,baolai,2013,1.6,手动,2013款 1.6L 手动 舒适型,model_31543_cs,dazhong,大众宝来,大众宝来 2013款 1.6L 手动 舒适型,dazhong
93,1450,11.98,baolai,2013,1.6,自动,2013款 1.6L 自动 时尚型,model_31545_cs,dazhong,大众宝来,大众宝来 2013款 1.6L 自动 时尚型,dazhong
94,1451,12.83,baolai,2013,1.6,自动,2013款 1.6L 自动 舒适型,model_31547_cs,dazhong,大众宝来,大众宝来 2013款 1.6L 自动 舒适型,dazhong
95,1452,13.63,baolai,2013,1.6,自动,2013款 1.6L 自动 豪华型,model_31546_cs,dazhong,大众宝来,大众宝来 2013款 1.6L 自动 豪华型,dazhong


In [2]:
cdi = pd.read_csv('../tmp/train/car_detail_info.csv')
cdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
id                     10 non-null int64
title                  10 non-null object
domain                 10 non-null object
predict_brand_slug     10 non-null int64
text                   10 non-null object
predict_model_slug     10 non-null float64
predict_detail_slug    8 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 640.0+ bytes


In [4]:
cdi

,id,title,domain,predict_brand_slug,text,predict_model_slug,predict_detail_slug
0,119065190,特斯拉-MODEL S 2015款 MODEL S 80D,renrenche.com,133,特斯拉models2015款models80d,2357.0,NaN
1,119068529,特斯拉-MODEL S 2015款 MODEL S 65D,renrenche.com,133,特斯拉models2015款models65d,2357.0,20612.0
2,119217629,特斯拉-MODEL S 2017款 MODEL S 80D,renrenche.com,133,特斯拉models2017款models80d,2357.0,30427.0
3,119724952,特斯拉-MODEL X 2017款 MODEL X 70D,renrenche.com,133,特斯拉modelx2017款modelx70d,2664.0,29113.0
4,120833258,特斯拉-MODEL S 2014款 MODEL S P65,renrenche.com,133,特斯拉models2014款modelsp65,2357.0,11053.0
5,120808043,特斯拉-MODEL S 2014款 MODEL S 65,renrenche.com,133,特斯拉models2014款models65,2357.0,18113.0
6,121002792,特斯拉-MODEL S 2015款 MODEL S 90D,renrenche.com,133,特斯拉models2015款models90d,2357.0,23334.0
7,122052781,特斯拉-MODEL S 2017款 MODEL S 80D,renrenche.com,133,特斯拉models2017款models80d,2357.0,30427.0
8,122744264,特斯拉-MODEL S 2015款 MODEL S 80D,renrenche.com,133,特斯拉models2015款models80d,2357.0,NaN
9,122977744,特斯拉-MODEL S 2015款 MODEL S 65D,renrenche.com,133,特斯拉models2015款models65d,2357.0,20612.0


In [24]:
test = pd.read_csv('../tmp/train/wait_predict.csv')
test = test.loc[(test['domain'].isin(['guazi.com','renrenche.com'])),:]
test = test.loc[(test['brand_slug'] == 'dazhong'),:]
test['predict_brand_slug'] = test['brand_slug']
test.to_csv('../tmp/train/predict.csv', index=False)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109175 entries, 898079 to 7704572
Data columns (total 7 columns):
id                    109175 non-null int64
title                 109175 non-null object
brand_slug            109175 non-null object
model_slug            109175 non-null object
model_detail_slug     109175 non-null object
domain                109175 non-null object
predict_brand_slug    109175 non-null object
dtypes: int64(1), object(6)
memory usage: 6.7+ MB


In [78]:
def delete_useless_char(df):
    text = df['value']
    text = text.replace('&nbsp','')
    text = text.replace('/','')
    text = text.replace(';','')
    return text

f = open("../tmp/train/new.car.info.json", encoding='utf-8')
count = 0
final = pd.DataFrame()
for text_json in f.readlines():
    detail = json.loads(text_json)
    config = detail.pop('config')
    param = detail.pop('param')
    specitems = detail.pop('specitems')
    if len(specitems) == 0:
        continue
    result = pd.DataFrame(specitems)
    result['brand'] = detail['brand']
    result['brand_id'] = detail['brand_id']
    result['type'] = detail['type']
    result['type_id'] = detail['type_id']
    result['manufacturer'] = detail['manufacturer']
    result = result.rename(columns={'id':'specid'})
    count = count + 1
    print(count,'combine',detail['type'])
    for i in range(0, len(param)):
        param_temp = pd.DataFrame(param[i])
        for j in range(0, len(param_temp)):
            paramitems = param_temp.loc[j,'paramitems']
            temp = pd.DataFrame(paramitems['valueitems'])
            temp = temp.rename(columns={'value':paramitems['name']})
            result = result.merge(temp, how='left', on=['specid'])

    for i in range(0, len(config)):
        config_temp = pd.DataFrame(config[i])
        for j in range(0, len(config_temp)):
            configitems = config_temp.loc[j,'configitems']
            temp = pd.DataFrame(configitems['valueitems'])
            temp['value'] = temp.apply(delete_useless_char,axis=1)
            temp = temp.rename(columns={'value':configitems['name']})
            result = result.merge(temp, how='left', on=['specid'])

    final = final.append(result,sort=False)
    final.reset_index(inplace=True, drop=True)
final.to_csv('../tmp/train/car_autohome_new.csv', index=False)

1 combine 奥迪A4L
2 combine 奥迪A6L
3 combine 奥迪A3
4 combine 奥迪A6L新能源
5 combine 奥迪Q2L
6 combine 奥迪Q3
7 combine 奥迪Q5L
8 combine 奥迪Q5
9 combine 奥迪RS 3
10 combine 奥迪RS 7
11 combine 奥迪RS 6
12 combine 奥迪R8
13 combine 奥迪A1
14 combine 奥迪TT RS
15 combine 奥迪A3(进口)
16 combine 奥迪S3
17 combine 奥迪A4(进口)
18 combine 奥迪S4
19 combine 奥迪A5
20 combine 奥迪S5
21 combine 奥迪S6
22 combine 奥迪A6(进口)
23 combine 奥迪S7
24 combine 奥迪A7
25 combine 奥迪A8
26 combine 奥迪Q7新能源
27 combine 奥迪Q7
28 combine 奥迪TTS
29 combine 奥迪A3新能源(进口)
30 combine 奥迪TT
31 combine 奥迪Q5(进口)
32 combine V8 Vantage
33 combine Rapide
34 combine V12 Vantage
35 combine 阿斯顿・马丁DB9
36 combine Vanquish
37 combine 阿斯顿・马丁DB11
38 combine 阿斯顿・马丁DBS
39 combine AC Schnitzer X5
40 combine Stelvio
41 combine Giulia
42 combine ALPINA B4
43 combine 奔驰C级
44 combine 奔驰E级
45 combine 奔驰GLA
46 combine 奔驰GLC
47 combine 奔驰V级
48 combine 威霆
49 combine 凌特
50 combine 奔驰A级(进口)
51 combine 奔驰B级
52 combine 奔驰CLA级
53 combine 奔驰C级(进口)
54 combine 奔驰CLS级
55 combine 奔驰E级(进口)
56 combine 奔驰GL

KeyboardInterrupt: 

In [102]:
def delete_useless_char(df):
    text = df['value']
    text = text.replace('&nbsp','')
    text = text.replace('/','')
    text = text.replace(';','')
    return text

f = open("../tmp/train/stop.sell.info.json", encoding='utf-8')
count = 0
final = pd.DataFrame()
for text_json in f.readlines():
    detail = json.loads(text_json)
    config = detail.pop('result')['configitems']

    result = pd.DataFrame([''],columns=['brand'])
    result['brand'] = detail['brand']
    result['brand_id'] = detail['brand_id']
    result['type'] = detail['type']
    result['type_id'] = detail['type_id']
    result['manufacturer'] = detail['manufacturer']
    result['stop_sell_name'] = detail['stop_sell_name']
    result['sell_type'] = detail['sell_type']
    result['id'] = detail['specid']

    count = count + 1
    print(count,'combine',detail['type'])

    for i in range(0, len(config)):
        config_temp = pd.DataFrame(config[i])
        for j in range(0, len(config_temp)):
            configitems = config_temp.loc[j,'items']
            temp = pd.DataFrame(configitems['modelexcessids'])
            temp['value'] = temp.apply(delete_useless_char,axis=1)
            temp = temp.rename(columns={'value':configitems['name']})
            result = result.merge(temp, how='left', on=['id'])

    final = final.append(result,sort=False)
    final.reset_index(inplace=True, drop=True)
final.to_csv('../tmp/train/car_autohome_stop.csv', index=False)

0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A3
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A3
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 combine 奥迪A4L
0 co

KeyboardInterrupt: 

In [104]:
def delete_useless_char(df):
    text = df['value']
    text = text.replace('&nbsp','')
    text = text.replace('/','')
    text = text.replace(';','')
    return text

f = open("../tmp/train/car.4s.price.json.true", encoding='utf-8')
count = 0
final = pd.DataFrame()
# for text_json in f.readlines():
text_json in f.readline()
detail = json.loads(text_json)
# config = detail.pop('result')['configitems']

# result = pd.DataFrame([''],columns=['brand'])
# result['brand'] = detail['brand']
# result['brand_id'] = detail['brand_id']
# result['type'] = detail['type']
# result['type_id'] = detail['type_id']
# result['manufacturer'] = detail['manufacturer']
# result['stop_sell_name'] = detail['stop_sell_name']
# result['sell_type'] = detail['sell_type']
# result['id'] = detail['specid']

# count = count + 1
# print(count,'combine',detail['type'])

# for i in range(0, len(config)):
#     config_temp = pd.DataFrame(config[i])
#     for j in range(0, len(config_temp)):
#         configitems = config_temp.loc[j,'items']
#         temp = pd.DataFrame(configitems['modelexcessids'])
#         temp['value'] = temp.apply(delete_useless_char,axis=1)
#         temp = temp.rename(columns={'value':configitems['name']})
#         result = result.merge(temp, how='left', on=['id'])

# final = final.append(result,sort=False)
# final.reset_index(inplace=True, drop=True)
detail
# final.to_csv('../tmp/train/car_4s_price.csv', index=False)

{'brand': '奥迪',
 'brand_id': '33',
 'manufacturer': '一汽-大众奥迪',
 'message': '',
 'result': {'bigpicprefix': '',
  'configitems': [{'items': [{'modelexcessids': [{'id': 19485,
        'value': '主● / 副●'}],
      'name': '主/副驾驶座安全气囊'},
     {'modelexcessids': [{'id': 19485, 'value': '前● / 后○'}],
      'name': '前/后排侧气囊'},
     {'modelexcessids': [{'id': 19485, 'value': '前● / 后●'}],
      'name': '前/后排头部气囊(气帘)'},
     {'modelexcessids': [{'id': 19485, 'value': '-'}], 'name': '膝部气囊'},
     {'modelexcessids': [{'id': 19485, 'value': '○'}], 'name': '胎压监测装置'},
     {'modelexcessids': [{'id': 19485, 'value': '-'}], 'name': '零胎压继续行驶'},
     {'modelexcessids': [{'id': 19485, 'value': '●'}], 'name': '安全带未系提示'},
     {'modelexcessids': [{'id': 19485, 'value': '●'}], 'name': 'ISOFIX儿童座椅接口'},
     {'modelexcessids': [{'id': 19485, 'value': '●'}], 'name': 'ABS防抱死'},
     {'modelexcessids': [{'id': 19485, 'value': '●'}],
      'name': '制动力分配(EBD/CBC等)'},
     {'modelexcessids': [{'id': 19485, 'value': '